taken from 
https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/Character_Level_RNN_Solution.ipynb

text can be found at:-
https://raw.githubusercontent.com/WillKoehrsen/deep-learning-v2-pytorch/master/recurrent-neural-networks/char-rnn/data/anna.txt

Import


In [22]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

Load

In [23]:

# open text file and read in data as `text`
with open('/content/drive/MyDrive/AnnaK.txt', 'r') as f:
    text = f.read()

In [24]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

Tokenisation

In [25]:

# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])


In [26]:
encoded

array([16, 75, 35, ...,  0, 76, 76])

Preprocess data

In [27]:

def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot


In [28]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


Create batches

In [29]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y


Test implementation

In [30]:

batches = get_batches(encoded, 64, 50)
x, y = next(batches)

In [31]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])


x
 [[16 75 35 33 61  3 37 25 20 76]
 [48 25 61 78 25 33 57 61 25 78]
 [48 78 34 26 61 73 75 79 25  5]
 [48  3 68  3 17 72 25 57 48  5]
 [68 79 25  5 75 78 76 35 17  5]
 [46 46 26 48 68 61 35 48 61 35]
 [61 70 26 48 28 25  5 26 61 75]
 [76 55 48 11 25 75  3 25 52  3]
 [11 25 35 48 11 25 78 72 25 68]
 [25 68 61 26 17 17 76 71 78 57]]

y
 [[75 35 33 61  3 37 25 20 76 76]
 [25 61 78 25 33 57 61 25 78 48]
 [78 34 26 61 73 75 79 25  5 75]
 [ 3 68  3 17 72 25 57 48  5 78]
 [79 25  5 75 78 76 35 17  5 35]
 [46 26 48 68 61 35 48 61 35 48]
 [70 26 48 28 25  5 26 61 75 25]
 [55 48 11 25 75  3 25 52  3 28]
 [25 35 48 11 25 78 72 25 68 78]
 [68 61 26 17 17 76 71 78 57 48]]


LSTM Input output

In [32]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')


Training on GPU!


In [33]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

Train

In [34]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))


Instantiate model

In [35]:

# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)


CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [36]:
batch_size = 64
seq_length = 80
n_epochs = 50 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)


Epoch: 1/50... Step: 10... Loss: 3.2384... Val Loss: 3.2225
Epoch: 1/50... Step: 20... Loss: 3.1414... Val Loss: 3.1405
Epoch: 1/50... Step: 30... Loss: 3.1349... Val Loss: 3.1280
Epoch: 1/50... Step: 40... Loss: 3.1382... Val Loss: 3.1214
Epoch: 1/50... Step: 50... Loss: 3.1060... Val Loss: 3.1207
Epoch: 1/50... Step: 60... Loss: 3.1173... Val Loss: 3.1178
Epoch: 1/50... Step: 70... Loss: 3.1212... Val Loss: 3.1157
Epoch: 1/50... Step: 80... Loss: 3.1045... Val Loss: 3.1107
Epoch: 1/50... Step: 90... Loss: 3.1098... Val Loss: 3.1013
Epoch: 1/50... Step: 100... Loss: 3.1029... Val Loss: 3.0895
Epoch: 1/50... Step: 110... Loss: 3.0546... Val Loss: 3.0426
Epoch: 1/50... Step: 120... Loss: 2.9636... Val Loss: 2.9465
Epoch: 1/50... Step: 130... Loss: 2.8486... Val Loss: 2.8192
Epoch: 1/50... Step: 140... Loss: 2.8102... Val Loss: 2.7699
Epoch: 1/50... Step: 150... Loss: 2.6545... Val Loss: 2.6463
Epoch: 1/50... Step: 160... Loss: 2.5839... Val Loss: 2.5711
Epoch: 1/50... Step: 170... Loss:

Checkpoint

In [37]:

# change the name, for saving multiple files
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)


Predict

In [38]:

def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h


Prime & generate text

In [39]:

def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)


In [40]:
print(sample(net, 1000, prime='Anna', top_k=5))


Anna
Alexandrovna, watching the doorway the propenty and strange and danking
in the high coat to be annoyed that the children at the same pirtunation of
his forgetting, the mare always did that the summer with which Anna
had trued to go out of the room with a brown, bushes and trinding their
chairs and brilliance of that stout, sounds, soft barrier, and the day
before had brought the staircase wondering who do then at least the social position, half
the steps, and the dark count of soup, the same peasant, the mother on
her shoulders, her eyes, as it was to her husband in which he had
been doing a good man, too, was standing at those eyes of the conversation,
would have said against the same station, and she had to do it, and with
the party in the sun, and a lady saw for the summer for a long thing,
which was not an unhappiness of ate that the world handed it on the possibility
of having throothe and more attractive. But as he shall believed
that this present provens of what they had su

Load a checkpoint

In [41]:

# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])


<All keys matched successfully>

In [42]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="Leven"))


Leven, and
his son's sea she went into the broad room where he sand over it, a men of
the same steps they were saying a club atmed on the right of a book, to
summer about the deciding of a common that had come into his sees that
they could never have spoken of it. He had all thought on him for
this delightful, and as though to be done, and that his companions
were considered anyone who was all the preparations of all the post,
the mirch of his own country, who depressed a companion of his superficial
condection. And therefore, had not called out whether he saw him.
Though the consciousness of her heart had been turned out. She was
delighted. She saw him what he had said to him, and saw her to show
him a ball the creath would not be significantly, and stopped. The peasant
was setted and shaked in first to the door, and a smile which, stirring
a silence. She lay straight bitter than something. A clat on the trouser
of hands, and the big, standing of a prepant departed serfun for the
thic